<a href="https://colab.research.google.com/github/fdevil420/test/blob/main/Diabetic_Retinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

In [ ]:
zip_file_path = '/content/drive/MyDrive/final year project/dataset/train.zip'
extract_path = '/content/sample_data'

In [ ]:
zip_file_path = '/content/drive/MyDrive/final year project/dataset/test.zip'
extract_path = '/content/sample_data'

In [ ]:
zip_file_path = '/content/drive/MyDrive/final year project/dataset/val.zip'
extract_path = '/content/sample_data'

In [ ]:
if not os.path.exists(zip_file_path):
    print(f"The file '{zip_file_path}' does not exist.")
else:
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Try to extract the contents of the zip file
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("Extraction successful.")
    except zipfile.BadZipFile:
        print("Error: The file is not a valid zip file.")

Extraction successful.


## **LIBRARIES**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os

### **DATASET**

In [2]:
file_paths = [
    ('/content/drive/MyDrive/final year project/dataset/train.zip', '/content/sample_data'),
    ('/content/drive/MyDrive/final year project/dataset/test.zip', '/content/sample_data'),
    ('/content/drive/MyDrive/final year project/dataset/val.zip', '/content/sample_data')
]


In [ ]:
for zip_file_path, extract_path in file_paths:
    if not os.path.exists(zip_file_path):
        print(f"The file '{zip_file_path}' does not exist.")
    else:
        # Create the extraction directory if it doesn't exist
        os.makedirs(extract_path, exist_ok=True)

        # Try to extract the contents of the zip file
        try:
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print("Extraction successful.")
        except zipfile.BadZipFile:
            print("Error: The file is not a valid zip file.")

In [ ]:
train =  '/content/sample_data/train'
val = '/content/sample_data/val'
test = '/content/sample_data/test'

### **MODEL**

In [ ]:
# Load ResNet152 model pre-trained on ImageNet data
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(255, 255, 3))

# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Create your own model by adding custom layers on top of the pre-trained model


In [ ]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # Adjust the output units based on the number of classes
])

In [ ]:
model.summary()


In [ ]:
seed_value = 42  # Seed for random transformations (change this value as needed)
np.random.seed(seed_value)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear intensity
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    vertical_flip=True,  # Random vertical flip
    fill_mode='nearest'  # Fill mode for handling points outside the input boundaries
)


val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:


# Example paths, replace with your actual dataset paths
train_data_dir = train
validation_data_dir = val

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
history = model.fit(train_generator, epochs=15, validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

In [ ]:
model.save("my_model.h5")

In [ ]:
# Make predictions
Y_pred = model.predict(validation_generator)
y_pred = tf.argmax(Y_pred, axis=1)

# Convert one-hot encoded labels to integers
y_true = validation_generator.classes

# Classification Report
print("\nClassification Report:\n", classification_report(y_true, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

# Calculate ROC curve and AUC-ROC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(5):  # Assuming 5 classes
    fpr[i], tpr[i], _ = roc_curve(y_true == i, Y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
for i in range(5):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for each class')
plt.legend(loc='lower right')
plt.show()

# Plot accuracy graph
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss graph
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()